In [11]:
import sys
import os
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())

import netCDF4 as nc
from netCDF4 import MFDataset
from os import listdir, system
from os.path import isfile, join, isdir
import numpy as np
import imp
import itertools
import os
import sys
import time
import inspect
import copy
from util import get_camfiles,convert_nc_data_to_tensor

In [ ]:
class DataSet(object):

    def __init__(self,
               images,
               labels,
               dtype=dtypes.float32):


        self._images = images
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0

    @property
    def images(self):
        return self._images

    @property
    def labels(self):
        return self._labels

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed
    
    def shuffle(self):
        pass

    def next_batch(self, batch_size, shuffle=True):
        """Return the next `batch_size` examples from this data set."""
        start = self._index_in_epoch
        # Shuffle for the first epoch
        if self._epochs_completed == 0 and start == 0 and shuffle:
            perm0 = np.arange(self._num_examples)
            np.random.shuffle(perm0)
            
            #shuffle files
            seed = np.random.randint()
            self._images.shuffle(seed)
            #todo: same for labels
            self._labels = self.labels[perm0]

        # Go to the next epoch
        if start + batch_size > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Get the rest examples in this epoch
            rest_num_examples = self._num_examples - start
            images_rest_part = self._images[start:self._num_examples]
            labels_rest_part = self._labels[start:self._num_examples]
            # Shuffle the data
            if shuffle:
                perm = np.arange(self._num_examples)
                np.random.shuffle(perm)
                self._images = self.images[perm]
                self._labels = self.labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size - rest_num_examples
            end = self._index_in_epoch
            images_new_part = self._images[start:end]
            labels_new_part = self._labels[start:end]
            return np.concatenate((images_rest_part, images_new_part), axis=0), np.concatenate((labels_rest_part, labels_new_part), axis=0)
        else:
            self._index_in_epoch += batch_size
            end = self._index_in_epoch
            return self._images[start:end], self._labels[start:end]


In [2]:
class ClimateImage(object):
    def __init__(self,filepaths, variables=["TMQ", "VBOT", "PSL"],
                 time_step_sample_freq=2, time_steps_per_example=1,time_steps_per_file=8):
        

        frame = inspect.currentframe()
        # set self.k = v for every k,v pair in __init__ except self of course
        self.set_constructor_args(frame)
        
        self.num_files = len(self.filepaths)
        self.examples_per_file = (time_steps_per_file / time_step_sample_freq) / time_steps_per_example
        self.total_examples = self.num_files * self.examples_per_file

    def set_constructor_args(self,frame):
        #set data members for object from constructor args
        _, _, _, params = inspect.getargvalues(frame)
        del params["frame"]
        for k,v in params.iteritems():
            setattr(self,k,v)
        
        
    
    def shuffle(self, seed):
        inds = np.arange(len(self.filepaths))
        np.random.RandomState(seed).shuffle(inds)
        '''only shufflez files'''
        self.filepaths=self.filepaths[inds]

    
    #overloading of bracket operators
    def __getitem__(self, slice_):
        slices = self.convert_slice_to_file_and_ex_inds(slice_)
        file_slice = slices["file_slice"]
        ex_slice = slices["ex_slice"]
        
        print file_slice
        print ex_slice
        filepaths = self.filepaths[file_slice]
        tens = self.grab_data_chunk(filepaths)
        images = tens[ex_slice]
        return images
    
    def convert_slice_to_file_and_ex_inds(self, slice_):
        if isinstance(slice_, slice):
            start, stop, step = [getattr(slice_,k) for k in ["start", "stop", "step"]]
            assert step==1 or step is None, "step must be 1 or None"
        
        elif isinstance(slice_, int):
            start, stop = [slice_, slice_]
            
        slices =  self.get_file_and_ex_inds(start, stop)
        return slices
        
        
        
            
    def get_file_and_ex_inds(self, start, stop):
        #file start stop indices to index filenames
        file_start, file_stop = self.get_file_ind(start), self.get_file_ind(stop)
        
        # get some useful numbers
        tot_examples_desired = stop - start 
        
        #relative example indices after examples read in
        ex_start = self.get_relative_ex_ind(start)
        ex_stop = ex_start + tot_examples_desired
        
        file_slice = slice(file_start,file_stop + 1)
        ex_slice = slice(ex_start,ex_stop)
        
        return {"file_slice":file_slice, "ex_slice": ex_slice}
    

    

    def get_file_ind(self,ex_ind):
        return ex_ind / self.examples_per_file

    def get_relative_ex_ind(self, ex_ind):
        return ex_ind % self.examples_per_file
        
        
        
    def grab_data_chunk(self, filepaths):
        """grabs input data (converts filepaths to np tensors)
        returns len(filepaths)*4, 16, 768,1152 array"""
        


        dataset=MFDataset(filepaths)
        
        tensor = convert_nc_data_to_tensor(dataset,self.variables, 
                                           self.time_step_sample_freq, self.time_steps_per_example)
 
        return tensor

In [7]:
# if __name__ == "__main__":
files = '''cam5_1_amip_run2.cam2.h2.1980-11-05-00000.nc
cam5_1_amip_run2.cam2.h2.1980-11-06-00000.nc
cam5_1_amip_run2.cam2.h2.1980-11-07-00000.nc
cam5_1_amip_run2.cam2.h2.1980-11-08-00000.nc
cam5_1_amip_run2.cam2.h2.1980-11-09-00000.nc
cam5_1_amip_run2.cam2.h2.1980-11-10-00000.nc
cam5_1_amip_run2.cam2.h2.1980-11-11-00000.nc
cam5_1_amip_run2.cam2.h2.1980-11-12-00000.nc
cam5_1_amip_run2.cam2.h2.1980-11-13-00000.nc
cam5_1_amip_run2.cam2.h2.1980-11-14-00000.nc
cam5_1_amip_run2.cam2.h2.1980-11-15-00000.nc
cam5_1_amip_run2.cam2.h2.1980-11-16-00000.nc'''.split("\n")

filepaths = [join("/home/evan/data/climate/input",fname) for fname in files]

ims =ClimateImage(filepaths, time_steps_per_example=1)

ex =ims[2:5]



slice(2, 6, None)
slice(0, 3, None)
